In [1]:
#!pip install --upgrade pip
#!pip install transformers==3.1.0

In [1]:
from transformers import pipeline
from os import listdir
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np

I0802 19:35:29.180007 139830866601792 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
I0802 19:35:29.203221 139830866601792 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [3]:
#classifier = pipeline("zero-shot-classification")
classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

I0802 19:36:13.281779 139830866601792 filelock.py:274] Lock 139825313352056 acquired on /root/.cache/torch/transformers/c2341a51039a311cb3c7dc71b3d21970e6a127876f067f379f8bcd77ef870389.6a09face0659d64f93c9919f323e2ad4543ca9af5d2417b1bfb1a36f2f6b94a4.lock


I0802 19:36:13.714730 139830866601792 filelock.py:318] Lock 139825313352056 released on /root/.cache/torch/transformers/c2341a51039a311cb3c7dc71b3d21970e6a127876f067f379f8bcd77ef870389.6a09face0659d64f93c9919f323e2ad4543ca9af5d2417b1bfb1a36f2f6b94a4.lock


I0802 19:36:14.134128 139830866601792 filelock.py:274] Lock 139825313352056 acquired on /root/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


I0802 19:36:14.868336 139830866601792 filelock.py:318] Lock 139825313352056 released on /root/.cache/torch/transformers/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


I0802 19:36:15.289589 139830866601792 filelock.py:274] Lock 139825313055688 acquired on /root/.cache/torch/transformers/414816afc2ab8922d082f893dbf90bcb9a43f09838039249c6c8ca3e8b77921f.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


I0802 19:36:15.765900 139830866601792 filelock.py:318] Lock 139825313055688 released on /root/.cache/torch/transformers/414816afc2ab8922d082f893dbf90bcb9a43f09838039249c6c8ca3e8b77921f.455d944f3d1572ab55ed579849f751cf37f303e3388980a42d94f7cd57a4e331.lock


I0802 19:36:16.298171 139830866601792 filelock.py:274] Lock 139825312594072 acquired on /root/.cache/torch/transformers/3efcb155a9475fe6b9318b8a8d5278bce1972d30291f97f2a8faeb50d02acabc.087b9fac49619019e540876a2d8ecb497884246b5aa8c9e8b7a0292cfbbe7c52.lock


I0802 19:36:40.389094 139830866601792 filelock.py:318] Lock 139825312594072 released on /root/.cache/torch/transformers/3efcb155a9475fe6b9318b8a8d5278bce1972d30291f97f2a8faeb50d02acabc.087b9fac49619019e540876a2d8ecb497884246b5aa8c9e8b7a0292cfbbe7c52.lock


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [247]:
#path_screenplays_scenes='./transcripts/shot-aligned_transcripts_2021_Max_Jack_Tanya.csv'
path_screenplays_scenes='./transcripts/shot-aligned_transcripts_2021_Peggy_Archie.csv'





In [233]:


def get_results(df, classifier, topic_labels):
    df['topic'] = None
    df['score'] = None
    
    for i in tqdm(df.index):
        text = df.loc[i, 'transcript']
        if text is not None:
            result = classifier(text, topic_labels)
            #df.loc[i, 'topic'] = str(str(result['labels'][0])+"+"+str(result['labels'][1]))
            #df.loc[i, 'score'] = str(str(result['scores'][0])+"+"+str(result['scores'][1]) 
            df.loc[i, 'topic'] = result['labels'][0]
            df.loc[i, 'score'] = result['scores'][0]
            #if result['labels'][0]=='surprise':
                #df.loc[i, 'score_hf'] = result['scores'][0]
            #else:
                #df.loc[i, 'score_hf'] = -1
            
    return df


    
    
    

    




In [248]:
candidate_labels_df=pd.read_csv('life_events_scale.csv',sep='\t', names=['topic','weight'])
candidate_labels_df=candidate_labels_df


candidate_labels=dict(zip(candidate_labels_df.topic, candidate_labels_df.weight))
for topic,weight in tqdm(candidate_labels.items()):
    print(topic)
    print(weight)
    


100%|██████████| 20/20 [00:00<00:00, 6554.11it/s]

Death relative
100
Divorce
73
Marital separation
65
Imprisonment
63
injury illness
53
Marriage
50
Fired
47
Marital reconciliation
45
Retirement
45
Pregnancy
40
Sexual difficulties
39
New family member
39
Business readjustment
39
Money change
38
Work change
36
Arguing
35
Mortgage
32
Child leaving home
29
Trouble with in-laws
29
Achievement
28


In [223]:
#soap_opera_scale={'extramarital affair': 1/1.98, 'get divorced': 1/1.96,'illegitimate child': 1/1.45,'institutionalized for emotional problem': 1/1.43,'happily married': 1/4.05,'serious accident': 1/2.96,'murdered': 1/1.81,'attempt suicide': 1/1.26,'blackmailed': 1/1.86,'unfaithful spouse': 1/2.23,'sexually assaulted': 1/2.60,'abortion': 1/1.41}
#candidate_labels={'death relative':100}

In [249]:
def get_results_several_binary(df, classifier, topic_labels):
    for topic,weight in tqdm(topic_labels.items()):
        df[topic] = None
        for i in df.index:
            text = df.loc[i, 'transcript']
            if text is not None:
                result = classifier(text,topic,multi_class=False)
                df.loc[i, topic] = result['scores'][0]*weight
        #df.to_csv('death_relative_Max_Jack_Tanya.csv')
        df.to_csv('life_scale_ranking_end_Peggy_Archie.csv')
        

            
    return df

In [250]:
df=pd.read_csv(path_screenplays_scenes)
df_with_text=df.dropna()
print(df_with_text.head())

    Unnamed: 0        begin          end                 filename  shot_id  \
5            5  00:00:26:14  00:00:27:16  5300410550331962313.xml        6   
8            8  00:00:31:15  00:00:35:05  5300410550331962313.xml        9   
9            9  00:00:35:06  00:00:36:12  5300410550331962313.xml       10   
12          12  00:00:38:16  00:00:42:24  5300410550331962313.xml       13   
13          13  00:00:43:00  00:00:45:14  5300410550331962313.xml       14   

                                           transcript  
5                  I think you'll find that's for me!  
8   You're not gonna get a tip,  if that's what yo...  
9                      Just sign here, love, please.   
12  Well, I know that outfit  ain't for my benefit...  
13  Why do men think everything  revolves around t...  


In [251]:
#candidate_labels=['drama']
first_results=get_results_several_binary(df_with_text,classifier,candidate_labels)
#first_results=get_results(df_with_text,classifier,candidate_labels)
cols_to_keep=['Unnamed: 0', 'begin', 'end', 'filename', 'shot_id', 'transcript']
first_results['score'] = first_results.drop(cols_to_keep, axis=1).max(axis=1)
first_results=first_results.sort_values(by=['score'],ascending=False)

#first_results.to_csv('death_relative_Max_Jack_Tanya.csv')
first_results.to_csv('life_scale_ranking_Peggy_Archie.csv')

print(first_results)
    
    


  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
100%|██████████| 20/20 [7:33:03<00:00, 1359.18s/it]  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

       Unnamed: 0        begin          end                 filename  shot_id  \
2169         2169  00:27:14:11  00:27:17:18  5300433743159556513.xml      476   
15429       15429  01:51:44:15  01:51:49:11  5318609185758554501.xml     1968   
18172       18172  02:22:33:07  02:22:37:09  5321170704253895214.xml     2688   
8949         8949  01:01:49:22  01:01:53:07  5310822839547632763.xml     1122   
17753       17753  01:59:58:22  02:00:03:22  5321170704253895214.xml     2269   
14716       14716  01:11:16:24  01:11:19:03  5318609185758554501.xml     1255   
11016       11016  00:58:25:02  00:58:30:08  5313398531435045858.xml     1072   
18990       18990  00:41:03:04  00:41:06:05  5323778608396028180.xml      725   
4203         4203  00:02:11:10  00:02:13:09  5305617339184877785.xml       22   
19977       19977  01:37:50:18  01:37:53:12  5323778608396028180.xml     1712   
18309       18309  00:03:07:02  00:03:09:05  5323778608396028180.xml       44   
17069       17069  01:24:08:

In [252]:

f1_scores=[]
#eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    candidate_labels=['investigation']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=joy,sadness, anger,fear, trust,distrust, surprise,anticipation
    #candidate_labels=['surprise','anticipation']
    #candidate_labels=['thought-provoking']
    
    #candidate_labels=['Action','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Mystery','Romance','SciFi','Thriller']
    #candidate_labels=['Action','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Romance','SciFi','Thriller']
    #candidate_labels=['anger', 'sadness','joy',feat]
    first_results=get_results(df,classifier,candidate_labels)
    first_results[['prediction_score','score_hf']]=MinMaxScaler().fit_transform(first_results[['prediction_score','score_hf']])
                  
    #print(first_results)
    #first_results['sum_score']=0.5*first_results['score_hf']+0.5*first_results['prediction_score']
    #print(first_results['sum_score'])
    #ranked=first_results[first_results.topic=='murder'].sort_values(by=['score'],ascending=False)
    first_results['prediction_class']=0
    #ranked['prediction_class']=0
    #ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #predicted_pos= ranked['scene_id'][ranked['prediction_class']==1]
    #predicted_pos=list(predicted_pos)
    #print(predicted_pos)
    first_results=first_results.sort_values(by=['score_hf'],ascending=False)
    #first_results=first_results.sort_values(by=['score_hf'],ascending=False)
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #first_results.to_csv('combined_scores_kill'+ep+'.csv',index=None)
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    f1_scores.append(f1_score(first_results.in_summary, first_results.prediction_class))
    
    
    

    
    
    

In [6]:
#print(first_results[first_results.in_summary==1])


print(first_results[['aspects','in_summary','topic','score','prediction_class','sum_score','prediction','prediction_score']])



NameError: name 'first_results' is not defined

In [40]:
print(np.mean(f1_scores))

print(len(f1_scores))



0.2940651856941449
39


In [ ]:
f1_scores_bis=[]





for ep in eps:
    df=pd.read_csv('combined_scores_murders'+ep+'.csv')
    #df['max']=df[["prediction_score", "score_hf"]].max(axis=1)
    df['max']=0.5*df['score_hf']+0.5*df['prediction_score']
    #df['max'][df.topic=='joy']=0
    df=df.sort_values(by=['prediction_score'],ascending=False)
    df['prediction_class']=0
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #print(df[['in_summary','topic','score_hf','prediction_class','sum_score','prediction','prediction_score','max']])
    
            
            
    #first_results.to_csv('combined_scores'+ep+'.csv')
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    df=df.sort_values(by=['score_hf'],ascending=False)
    df['prediction_class'].iloc[0:len(df[df.in_summary==1])]=1
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results0
    f1_scores_bis.append(f1_score(df.in_summary, df.prediction_class))
    


print(np.mean(f1_scores_bis))

#print(len(f1_scores))




In [6]:

f1_scores_emotions=[]
for ep in eps:
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    candidate_labels=['anger', 'anticipation', 'joy', 'trust', 'fear', 'surprise', 'sadness', 'disgust']
    first_results=get_results(df,classifier,candidate_labels)   
    first_results['prediction_class']=0
    first_results[first_results.topic=='Crime'].sort_values(by=['score'],ascending=False)[]

    ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    ranked.to_csv('murder_hum_class'+ep+'.csv')
    print(ranked)
    f1_scores_emotions.append(f1_score(ranked.in_summary, ranked.prediction_class))
   
classifier(sequence, candidate_labels, multi_class=True)


SyntaxError: invalid syntax (<ipython-input-6-c61821b32eb2>, line 9)

In [114]:
import numpy as np

sorted_f1 = f1_scores.copy()
sorted_f1.sort()
print(f1_scores[10])
print(eps[10])
ep=eps[10]
print(sorted_f1)
df=pd.read_csv('murder_hum_class'+ep+'.csv')

#print(df.scene_text[df.in_summary==0].iloc[4])
print(df)


1.0
s04e09
[0.19999999999999998, 0.2857142857142857, 0.28571428571428575, 0.3157894736842105, 0.3333333333333333, 0.36363636363636365, 0.4, 0.4, 0.4285714285714285, 0.43478260869565216, 0.45454545454545453, 0.47058823529411764, 0.5, 0.5, 0.5, 0.5, 0.5000000000000001, 0.5714285714285714, 0.6, 0.6, 0.6, 0.6153846153846153, 0.6153846153846153, 0.6153846153846153, 0.631578947368421, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.7000000000000001, 0.7142857142857143, 0.7142857142857143, 0.7272727272727273, 0.7272727272727273, 0.7272727272727273, 0.7499999999999999, 0.7499999999999999, 0.8571428571428571, 1.0]
   Unnamed: 0  Unnamed: 0.1  scene_id  \
0           8             8         8   
1          30            30        30   

                                          scene_text  in_summary      aspects  \
0  [' ( Scene opens on a close up of two prints ,...           1  Death cause   
1  [' ( CATHERINE puts the bagged vice grips on t...           1   

In [4]:
df_all=pd.DataFrame(columns=['Unnamed: 0', 'scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction_score', 'prediction', 'aspect?'])
f1_scores=[]
eps=['s05e10']
for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    print(df.columns)
    df_all=df_all.append(df)
    
   


s05e10
Index(['Unnamed: 0', 'scene_id', 'scene_text', 'in_summary', 'aspects',
       'prediction_score', 'prediction', 'aspect?'],
      dtype='object')


In [ ]:

eps=[]

for ep in listdir(path_screenplays_scenes):
    annotated_scenes=pd.read_csv(path_screenplays_scenes+'/'+ep)
    eps.append(ep.split('.csv')[0])

print(eps)
    
tags_entire_episode=[]


eps=['s03e19', 's04e14', 's04e06', 's01e07', 's04e21', 's05e17', 's03e08', 's04e10', 's04e05', 's05e05', 's05e06', 's01e20', 's04e15', 's02e06', 's01e08', 's04e12', 's05e08', 's03e05', 's02e10', 's05e13', 's03e11', 's01e13']

for ep in eps:
    print(ep)
    df=pd.read_csv(ep+'_'+method_used+'_similarity.csv')
    #print(df)
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['kill','love']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy','murder']
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=joy,sadness, anger,fear, trust,distrust, surprise,anticipation
    candidate_labels=['murder','violence', 'flashback', 'romantic', 'cult', 'revenge', 'psychedelic', 'comedy', 'suspenseful', 'good_versus_evil', 'humor', 'satire', 'entertaining', 'neo noir', 'action', 'sadist', 'insanity', 'tragedy', 'fantasy', 'paranormal', 'boring', 'mystery', 'horror', 'melodrama', 'cruelty', 'gothic', 'dramatic', 'dark', 'atmospheric', 'sci-fi', 'psychological', 'historical', 'absurd', 'prank', 'sentimental', 'philosophical', 'avant garde', 'bleak', 'alternate reality', 'depressing', 'plot twist', 'realism', 'cute', 'stupid', 'intrigue', 'pornographic', 'home movie', 'haunting', 'historical fiction', 'allegory', 'adult comedy', 'inspiring', 'anti war', 'comic', 'brainwashing', 'alternate history', 'queer', 'clever', 'claustrophobic', 'whimsical', 'feel-good', 'blaxploitation', 'western', 'grindhouse film', 'magical realism', 'suicidal', 'autobiographical', 'christian film', 'non fiction']
    df['topic'] = None
    for i in tqdm(range(0, df.shape[0])):
        df.loc[i, 'topic']='bigoudi'
        text = df.loc[i, 'scene_text']
        result= classifier(text, 'murder')
        if text is not None:
            for j in candidate_labels:
                result_provisoire = classifier(text, j)
                #print(result_provisoire['scores'][0].type)
                #print(result_provisoire['scores'][0])
                #print(result['scores'][0])
                if result_provisoire['scores'][0]>result['scores'][0]:
                    result['scores']=result_provisoire['scores']
                    #print(result['scores'])
                    df.loc[i, 'topic'] = result_provisoire['labels'][0]
                    #print(df.loc[i, 'topic'])                   
            #print(result['labels'][0])
    #print(df.loc[i, 'topic'])
    print(df['topic'].value_counts().argmax())
    tags_entire_episode.append(df['topic'].value_counts().argmax())
    
    
   

In [ ]:


    sequence=df.scene_text
    #candidate_labels = ['Crime scene', 'Victim', 'Death cause', 'Perpetrator', 'Evidence', 'Motive']
    #candidate_labels=['crime scene', 'victim', 'death cause', 'perpetrator', 'evidence', 'motive']
    #candidate_labels=['kill']
    #action, drama, horror and romance.
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=['action', 'crime','romance','comedy']
    #candidate_labels=joy,sadness, anger,fear, trust,distrust, surprise,anticipation
    candidate_labels=['surprise','anticipation']
    
    #candidate_labels=['Action','Animation','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Mystery','Romance','SciFi','Thriller']
    #candidate_labels=['Action','Biography','Comedy','Crime','Drama','Family','Fantasy','Horror','Romance','SciFi','Thriller']
    #candidate_labels=['anger', 'sadness','joy',feat]
    first_results=get_results(df,classifier,candidate_labels)
    first_results[['prediction_score','score_hf']]=MinMaxScaler().fit_transform(first_results[['prediction_score','score_hf']])
                  
    #print(first_results)
    first_results['sum_score']=0.5*first_results['score_hf']+0.5*first_results['prediction_score']
    #print(first_results['sum_score'])
    #ranked=first_results[first_results.topic=='murder'].sort_values(by=['score'],ascending=False)
    first_results['prediction_class']=0
    #ranked['prediction_class']=0
    #ranked['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #predicted_pos= ranked['scene_id'][ranked['prediction_class']==1]
    #predicted_pos=list(predicted_pos)
    #print(predicted_pos)
    #first_results=first_results.sort_values(by=['sum_score'],ascending=False)
    first_results=first_results.sort_values(by=['score_hf'],ascending=False)
    first_results['prediction_class'].iloc[0:len(first_results[first_results.in_summary==1])]=1
    #first_results.to_csv('combined_scores_kill'+ep+'.csv',index=None)
    #first_results.loc[first_results['scene_id'].isin(predicted_pos), 'prediction_huggingface'] = 1 
    #print(first_results)
    f1_scores.append(f1_score(first_results.in_summary, first_results.prediction_class))
    

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [ ]:
sequence = "Who are you voting for in 2020?"
candidate_labels = ["politics", "public health", "economics", "elections"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

Here's an example of sentiment classification: 

In [ ]:
sequence = "I hated this movie. The acting sucked."
candidate_labels = ["positive", "negative"]

classifier(sequence, candidate_labels)

{'labels': ['negative', 'positive'],
 'scores': [0.9916268587112427, 0.00837317667901516],
 'sequence': 'I hated this movie. The acting sucked.'}

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [ ]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.

#### Update: Zero-shot classification in 100 languages

Interested in using the pipeline for languages other than English? We've trained a cross-lingual model on top of XLM RoBERTa which you can use by passing `model='joeddav/xlm-roberta-large-xnli'` when creating the pipeline:

In [ ]:
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

You can use it with any combination of languages. For example, let's classify a Russian sentence with English candidate labels:

In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics"]

classifier(sequence, candidate_labels)

{'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.9048484563827515, 0.05722189322113991, 0.03792969882488251],
 'sequence': 'За кого вы голосуете в 2020 году?'}

Now let's do the same but with the labels in French:



In [ ]:
sequence = "За кого вы голосуете в 2020 году?" # translation: "Who are you voting for in 2020?"
candidate_labels = ["Europe", "santé publique", "politique"]

classifier(sequence, candidate_labels)

{'labels': ['politique', 'Europe', 'santé publique'],
 'scores': [0.9726154804229736, 0.017128489911556244, 0.010256024077534676],
 'sequence': 'За кого вы голосуете в 2020 году?'}

As we discussed in the last section, the default hypothesis template is the English, `This text is {}.`. If you are working strictly within one language, it may be worthwhile to translate this to the language you are working with:

In [ ]:
sequence = "¿A quién vas a votar en 2020?"
candidate_labels = ["Europa", "salud pública", "política"]
hypothesis_template = "Este ejemplo es {}."

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'labels': ['política', 'Europa', 'salud pública'],
 'scores': [0.9109585881233215, 0.05954807624220848, 0.029493311420083046],
 'sequence': '¿A quién vas a votar en 2020?'}

The model is fine-tuned on XNLI which includes 15 languages: Arabic, Bulgarian, Chinese, English, French, German, Greek, Hindi, Russian, Spanish, Swahili, Thai, Turkish, Urdu, and Vietnamese. The base model is trained on 85 more, so the model will work to some degree for any of those in the XLM RoBERTa training corpus (see the full list in appendix A of the [XLM Roberata paper](https://arxiv.org/abs/1911.02116)).

See the [model page](https://huggingface.co/joeddav/xlm-roberta-large-xnli) for more.